In [6]:
import geopandas as gpd
import numpy as np 
import matplotlib.pyplot as plt
import pandas as pd
from shapely import wkt, Point

In [9]:
df = pd.read_csv('ads24-roads.csv', low_memory=False)
df = df.head(5)
df

,id,name,road_class,lanes,width,oneway,nodes,wkt
0,4911883,9 Maja,secondary,NaN,NaN,yes,"{1706340509,11956119625,11956119626,1195611962...",LINESTRING(19.143442800000003 51.5878166000000...
1,4911884,NaN,secondary,2,NaN,NaN,"{32127316,4703618827,538482267}","LINESTRING(19.1776191 51.601251700000006,19.17..."
2,4911885,NaN,secondary,2,NaN,NaN,"{31941772,32127316}","LINESTRING(19.1781495 51.601463700000004,19.17..."
3,4918113,NaN,tertiary_link,1,NaN,yes,"{32216651,300116154,10294036067,10294036070,10...","LINESTRING(21.036622400000002 52.1637064,21.03..."
4,4918117,NaN,tertiary,1,NaN,yes,"{32216708,300116160,3391231161,10293604891,300...","LINESTRING(21.0365343 52.162990300000004,21.03..."


In [10]:
df.fillna({'oneway': 'no'}, inplace=True)
df['nodes'] = df['nodes'].apply(lambda x: list(eval('[' + x[1:-1] + ']')))
df['start_node_id'] = df['nodes'].apply(lambda x: x[0])
df['end_node_id'] = df['nodes'].apply(lambda x: x[-1])
df['geometry'] = df['wkt'].apply(wkt.loads)
df['coordinates'] = df['geometry'].apply(lambda geom: list(geom.coords))

gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry='geometry')
gdf['long'] = gdf.centroid.x
gdf['lat'] = gdf.centroid.y

gdf.to_crs(epsg=2180, inplace=True)
gdf = pd.concat([gdf, gdf.bounds], axis=1)
df_roads = gdf.drop(['wkt', 'geometry', 'coordinates'], axis=1)
df_roads

/tmp/ipykernel_143536/2949358504.py:9: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['long'] = gdf.centroid.x
/tmp/ipykernel_143536/2949358504.py:10: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


,id,name,road_class,lanes,width,oneway,nodes,start_node_id,end_node_id,long,lat,minx,miny,maxx,maxy
0,4911883,9 Maja,secondary,NaN,NaN,yes,"[1706340509, 11956119625, 11956119626, 1195611...",1706340509,1706340508,19.143590,51.587732,509934.625969,413471.444445,509955.059355,413490.116246
1,4911884,NaN,secondary,2,NaN,no,"[32127316, 4703618827, 538482267]",32127316,538482267,19.177001,51.601004,512212.610609,414933.640287,512297.993234,414989.049071
2,4911885,NaN,secondary,2,NaN,no,"[31941772, 32127316]",31941772,32127316,19.177884,51.601358,512297.993234,414989.049071,512334.659612,415012.708960
3,4918113,NaN,tertiary_link,1,NaN,yes,"[32216651, 300116154, 10294036067, 10294036070...",32216651,32216658,21.036676,52.163553,639255.607407,479432.948067,639262.229480,479466.904609
4,4918117,NaN,tertiary,1,NaN,yes,"[32216708, 300116160, 3391231161, 10293604891,...",32216708,32124429,21.036949,52.163086,639251.819918,479384.056693,639309.026166,479406.925914


In [11]:
df_nodes = df[['id', 'nodes', 'coordinates']]
df_nodes['road_id'] = df_nodes['id']
df_nodes = df_nodes.explode(['nodes', 'coordinates'])

df_nodes['id'] = df_nodes['nodes'].astype(int)
df_nodes['geometry'] = df_nodes['coordinates'].apply(lambda coord: Point(coord[0], coord[1]))

gdf_nodes = gpd.GeoDataFrame(
    df_nodes[['id', 'geometry', 'road_id']],
    crs="EPSG:4326"
)

gdf_nodes['long'] = gdf_nodes.geometry.x
gdf_nodes['lat'] = gdf_nodes.geometry.y

gdf_nodes = gdf_nodes.to_crs(epsg=2180)
gdf_nodes['x'] = gdf_nodes.geometry.x
gdf_nodes['y'] = gdf_nodes.geometry.y

gdf_nodes = gdf_nodes.drop(['geometry'], axis=1)
gdf_nodes.drop_duplicates(subset="id", inplace=True)
gdf_nodes

/tmp/ipykernel_143536/72380530.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nodes['road_id'] = df_nodes['id']


,id,road_id,long,lat,x,y
0,1706340509,4911883,19.143443,51.587817,509934.625969,413490.116246
0,11956119625,4911883,19.143487,51.587796,509937.705646,413487.776349
0,11956119626,4911883,19.143537,51.587770,509941.174243,413484.892425
0,11956119627,4911883,19.143578,51.587737,509943.979359,413481.295636
0,11690955571,4911883,19.143649,51.587692,509948.948099,413476.302206
0,1706340508,4911883,19.143737,51.587648,509955.059355,413471.444445
1,32127316,4911884,19.177619,51.601252,512297.993234,414989.049071
1,4703618827,4911884,19.176947,51.600983,512251.537658,414959.050587
1,538482267,4911884,19.176384,51.600755,512212.610609,414933.640287
2,31941772,4911885,19.178149,51.601464,512334.659612,415012.708960


In [8]:
df.fillna({'oneway': 'no'}, inplace=True)
exploded_df = df[['oneway', 'nodes', 'id']].explode("nodes")
exploded_df["node_end"] = exploded_df["nodes"].shift(-1)
exploded_df["id_end"] = exploded_df["id"].shift(-1)
exploded_df = exploded_df[exploded_df['id'] == exploded_df['id_end']]
exploded_df = exploded_df.drop(columns=["id", 'id_end'])

exploded_df = exploded_df.dropna(subset=["node_end"])
exploded_df["node_start"] = exploded_df["nodes"]
exploded_df = exploded_df.drop(columns=["nodes"])
reverse_df = exploded_df[exploded_df["oneway"] == "no"].copy()
reverse_df[["node_start", "node_end"]] = reverse_df[["node_end", "node_start"]].values

road_node_connetions_df = pd.concat([exploded_df, reverse_df], ignore_index=True)
del exploded_df
del reverse_df
road_node_connetions_df.drop(columns=["oneway"], inplace=True)
road_node_connetions_df.drop_duplicates(subset=["node_end", "node_start"], inplace=True)
road_node_connetions_df


,node_end,node_start
0,11956119625,1706340509
1,11956119626,11956119625
2,11956119627,11956119626
3,11690955571,11956119627
4,1706340508,11690955571
5,4703618827,32127316
6,538482267,4703618827
7,32127316,31941772
8,300116154,32216651
9,10294036067,300116154


In [4]:
df.iloc[0]['nodes']

[1706340509, 11956119625, 11956119626, 11956119627, 11690955571, 1706340508]

In [3]:
df['nodes'] = df['nodes'].apply(lambda x: list(eval('[' + x[1:-1] + ']')))

df['geometry'] = df['wkt'].apply(wkt.loads)

df['coordinates'] = df['geometry'].apply(lambda geom: list(geom.coords))


In [18]:
df.iloc[619]['nodes']

[33391591, 5700445722, 5700445723, 33391592, 33391591]

In [19]:
len(df.iloc[619]['nodes']),len(df.iloc[619]['coordinates'])

(5, 5)

In [20]:
gdf = gpd.GeoDataFrame(df, crs="EPSG:4326", geometry='geometry')
gdf['long'] = gdf.centroid.x
gdf['lat'] = gdf.centroid.y
print('finished')

df_roads = gdf.drop(['wkt', 'geometry', 'coordinates', 'nodes'], axis=1)
df_roads

finished


/tmp/ipykernel_400504/4246636594.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['long'] = gdf.centroid.x
/tmp/ipykernel_400504/4246636594.py:3: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  gdf['lat'] = gdf.centroid.y


,id,name,road_class,lanes,width,oneway,long,lat
0,4911883,9 Maja,secondary,NaN,NaN,yes,19.143590,51.587732
1,4911884,NaN,secondary,2,NaN,yes,19.177001,51.601004
2,4911885,NaN,secondary,2,NaN,yes,19.177884,51.601358
3,4918113,NaN,tertiary_link,1,NaN,yes,21.036676,52.163553
4,4918117,NaN,tertiary,1,NaN,yes,21.036949,52.163086
...,...,...,...,...,...,...,...,...
9995,26533991,Generała Jana Henryka Dąbrowskiego,secondary,2,NaN,yes,16.696128,52.282193
9996,26533994,Bratysławska,tertiary,1,NaN,yes,19.938057,50.085951
9997,26533996,Biała,residential,NaN,NaN,yes,19.939897,50.089443
9998,26534608,NaN,service,NaN,NaN,yes,19.939910,50.092025


In [21]:
df_roads.isna().sum()

id               0
name          2532
road_class       0
lanes         5741
width         9850
oneway           0
long             0
lat              0
dtype: int64

In [ ]:
# df.fillna({'oneway': 'no'}, inplace=True)
# exploded_df = df[['oneway', 'nodes', 'id']].explode("nodes")
# exploded_df["node_end"] = exploded_df["nodes"].shift(-1)
# exploded_df["id_end"] = exploded_df["id"].shift(-1)
# exploded_df = exploded_df[exploded_df['id'] == exploded_df['id_end']]
# exploded_df = exploded_df.drop(columns=["id", 'id_end'])
# exploded_df

,oneway,nodes,node_end
0,yes,1706340509,11956119625
0,yes,11956119625,11956119626
0,yes,11956119626,11956119627
0,yes,11956119627,11690955571
0,yes,11690955571,1706340508
...,...,...,...
2588733,no,32126055,9556706623
2588733,no,9556706623,9556706622
2588733,no,9556706622,9556706621
2588733,no,9556706621,9556706620


In [7]:
df.fillna({'oneway': 'no'}, inplace=True)
exploded_df = df[['oneway', 'nodes', 'id']].explode("nodes")
exploded_df["node_end"] = exploded_df["nodes"].shift(-1)
exploded_df["id_end"] = exploded_df["id"].shift(-1)
exploded_df = exploded_df[exploded_df['id'] == exploded_df['id_end']]
exploded_df = exploded_df.drop(columns=["id", 'id_end'])

exploded_df = exploded_df.dropna(subset=["node_end"])
exploded_df["node_start"] = exploded_df["nodes"]
exploded_df = exploded_df.drop(columns=["nodes"])
reverse_df = exploded_df[exploded_df["oneway"] == "no"].copy()
reverse_df[["node_start", "node_end"]] = reverse_df[["node_end", "node_start"]].values

road_node_connetions_df = pd.concat([exploded_df, reverse_df], ignore_index=True)
del exploded_df
del reverse_df
road_node_connetions_df.drop(columns=["oneway"], inplace=True)
road_node_connetions_df.drop_duplicates(subset=["node_end", "node_start"], inplace=True)
road_node_connetions_df

,node_end,node_start
0,11956119625,1706340509
1,11956119626,11956119625
2,11956119627,11956119626
3,11690955571,11956119627
4,1706340508,11690955571
...,...,...
28809616,32126055,9556706623
28809617,9556706623,9556706622
28809618,9556706622,9556706621
28809619,9556706621,9556706620


In [10]:
# Create pairs of nodes


,oneway,node_end,node_start
0,yes,11956119625,1706340509
0,yes,11956119626,11956119625
0,yes,11956119627,11956119626
0,yes,11690955571,11956119627
0,yes,1706340508,11690955571
...,...,...,...
2588733,no,9556706623,32126055
2588733,no,9556706622,9556706623
2588733,no,9556706621,9556706622
2588733,no,9556706620,9556706621


In [ ]:
from shapely.geometry import Point

df_nodes = df[['id', 'nodes', 'coordinates']]
df_nodes['road_id'] = df_nodes['id']
df_nodes = df_nodes.explode(['nodes', 'coordinates'])

df_nodes['id'] = df_nodes['nodes'].astype(int)
df_nodes['geometry'] = df_nodes['coordinates'].apply(lambda coord: Point(coord[0], coord[1]))

gdf_nodes = gpd.GeoDataFrame(
    df_nodes[['id', 'geometry', 'road_id']],
    crs="EPSG:4326"
)

gdf_nodes['long'] = gdf_nodes.geometry.x
gdf_nodes['lat'] = gdf_nodes.geometry.y

gdf_nodes = gdf_nodes.to_crs(epsg=2180)
gdf_nodes['x'] = gdf_nodes.geometry.x
gdf_nodes['y'] = gdf_nodes.geometry.y

gdf_nodes = gdf_nodes.drop(['geometry'], axis=1)
gdf_nodes.drop_duplicates(subset="id", inplace=True)
gdf_nodes


KeyError: "['coordinates'] not in index"

In [ ]:
df_nodes['id'] = df_nodes['nodes'].astype(int)
df_nodes['geometry'] = df_nodes['coordinates'].apply(lambda coord: Point(coord[0], coord[1]))

gdf_nodes = gpd.GeoDataFrame(
    df_nodes[['id', 'geometry', 'road_id']],
    crs="EPSG:4326"
)

gdf_nodes['long'] = gdf_nodes.geometry.x
gdf_nodes['lat'] = gdf_nodes.geometry.y

gdf_nodes = gdf_nodes.to_crs(epsg=2180)
gdf_nodes['x'] = gdf_nodes.geometry.x
gdf_nodes['y'] = gdf_nodes.geometry.y

gdf_nodes = gdf_nodes.drop(['geometry'], axis=1)
gdf_nodes.drop_duplicates(subset="id", inplace=True)
gdf_nodes

In [23]:
gdf_nodes

,id,road_id,long,lat,x,y
0,1706340509,4911883,19.143443,51.587817,509934.625969,413490.116246
0,11956119625,4911883,19.143487,51.587796,509937.705646,413487.776349
0,11956119626,4911883,19.143537,51.587770,509941.174243,413484.892425
0,11956119627,4911883,19.143578,51.587737,509943.979359,413481.295636
0,11690955571,4911883,19.143649,51.587692,509948.948099,413476.302206
...,...,...,...,...,...,...
9999,290836186,26534966,21.567053,53.084764,671870.696923,583013.901245
9999,12149490887,26534966,21.567020,53.084789,671868.382373,583016.579059
9999,1130301488,26534966,21.566648,53.085068,671842.396534,583046.684165
9999,2996888310,26534966,21.566591,53.085113,671838.367006,583051.582623
